In [1]:
%run Dataset_cleanup/cleanup_clinical.py

Clinical data set imported


 The following features do not provide any information: 
 ['Composite.Element.REF' 'ethnicity' 'gender' 'pathologicstage'
 'pathologyMstage' 'tumortissuesite'] 



 Variables that are not known at initial diagnosis: 
 ['daystodeath' 'daystolastfollowup' 'daystopsa' 'gleasonscore'
 'histologicaltype' 'numberoflymphnodes' 'pathologyTstage'
 'radiationtherapy' 'residualtumor' 'vitalstatus'] 


Variables that are known at the time of diagnosis:
 ['dateofinitialpathologicdiagnosis' 'psavalue' 'race' 'yearstobirth']

Dimensions of clinical dataframe: (499, 7)


In [2]:
%run Dataset_cleanup/cleanup_gene_counts.py

Gene Counts data set imported

Dimension of DataFrame: (497, 20501)


## Benchmark Analysis of initial clinical presentation for  predictors of metastasis

In [3]:
"""The Clinical data (including metastasis label) have a multitude of missing items."""
print("Total observations in original dataset:",clinical.shape[0])

not_labeled = y[y.isnull()]
y_labels = y[y.notnull()]

print("\nLabeled observations:",y_labels.shape[0],"\nUnlabeled observations removed:",not_labeled.shape[0])


Total observations in original dataset: 499

Labeled observations: 426 
Unlabeled observations removed: 73


In [4]:
clinical = clinical.loc[y_labels.index]  #filter only observations where metastasis state is known
print("Filtering out missing metastasis state labels left",clinical.shape[0],"observations")

clinical.dropna(inplace = True) # Remove all observations where clinical data is missing
print("Removing NaN rows leaves ",clinical.shape[0]," observations in the feature set.") 

y_labels_NAdrop = y_labels.loc[clinical.index] #remove metastasis labels where no clinical data now exists due to NaN removal.

Filtering out missing metastasis state labels left 426 observations
Removing NaN rows leaves  359  observations in the feature set.


In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import recall_score
from sklearn.metrics import make_scorer, accuracy_score

In [6]:
recaller = make_scorer(recall_score, greater_is_better=True, needs_proba=False, needs_threshold=False, pos_label='n1')
accuracy = make_scorer(accuracy_score, greater_is_better=True, needs_proba=False, needs_threshold=False)
clf = DecisionTreeClassifier(criterion='gini',
                            splitter='best',
                            max_depth=None,
                            min_samples_split=30,
                            min_samples_leaf=1,
                            min_weight_fraction_leaf=0.0,
                            max_features=None,
                            random_state=None,
                            max_leaf_nodes=None,
                            class_weight='balanced', #avoids the classifier being rewarded for choosing the most prevelant class each instance (n0)
                            presort=False)
recall_vals = cross_val_score(clf, clinical, y_labels_NAdrop, scoring=recaller, cv=5)
accuracy_vals = cross_val_score(clf, clinical, y_labels_NAdrop, scoring=accuracy, cv=5)
print('Recall_n1:  ',recall_vals.mean())

print('\nModel Accuracy score:  ',accuracy_vals.mean())
print("\nNull error rate for metastasis:  ",(sum(y_labels == 'n1') / len(y_labels)))

Recall_n1:   0.465384615385

Model Accuracy score:   0.544853366776

Null error rate for metastasis:   0.18544600939


## Exploratory Analysis using Gene TPM counts as predictors of Metastasis

### Wrangle TPM counts DF

In [7]:
print(gene_counts.shape)
print(len(y_labels.index))
X = gene_counts.loc[list(y_labels.index)]  #Only observations that also have a known metastasis state are kept.  
print(X.shape)
print(sum(np.isfinite(X.iloc[:,1])))

(497, 20501)
426
(426, 20501)
424


In [8]:
X.dropna(axis=0, inplace=True)
print(X.shape[0])
y_labels_NA_gc_dropped = y_labels[X.index]
print(len(y_labels_NA_gc_dropped.index))

424
424


In [9]:
def duplicate_TCGA_ID_test(X) :
    seen = set()
    uniq = []
    duplicates = []
    for x in X.index:
        if x not in seen:
            uniq.append(x)
            seen.add(x)
        else:
            duplicates.append(x)
    if len(duplicates) > 0 :
        print("Error in data set arrangement")
    else :
        print("Data set contains",X.shape[0],"uniquely barcoded observations.")

duplicate_TCGA_ID_test(X)

Data set contains 424 uniquely barcoded observations.


### Feature Reduction

In [10]:
from sklearn.feature_selection import SelectKBest, f_classif

In [11]:
selector = SelectKBest(f_classif, k = 'all')
selector.fit(X, y_labels_NA_gc_dropped)

/Users/ccthomps/anaconda/lib/python3.5/site-packages/sklearn/feature_selection/univariate_selection.py:113: UserWarning: Features [ 1595  1815  2267  4324  4341  4342  4343  4344  4345  4346  4347  4779
  4780  4781  4782  4787  4790  4791  4794  4795  4801  4803  5259  6032
  6776  6780  6783  7445  7551  7632  7633  7634  7635  7636  7732  9275
  9277  9287  9291  9292  9419  9421  9423  9495 10092 11101 11929 12028
 12488 12527 12584 12593 12599 12601 12606 12635 12692 13749 13963 14015
 14129 14130 14131 14132 14726 14727 14729 15109 15111 15112 16536 16538
 16539 16541 16542 16543 16545 16546 16547 16548 16549 16550 16576 16605
 16608 16648 16668 16669 16670 16671 16672 16673 16674 16675 16676 16677
 16678 16679 16680 16681 16682 16683 16684 16685 16686 16687 16688 16689
 16690 16691 16692 16693 16694 16695 16696 16697 16698 16699 16700 16701
 16702 16703 16704 16705 16706 16707 16708 16709 16710 16711 16712 16713
 16714 16715 16716 16717 16718 16720 16721 16722 16723 16724 16725 

SelectKBest(k='all', score_func=<function f_classif at 0x113ceb510>)

In [12]:
GeneRanks= pd.DataFrame({'Gene':X.columns,'scores':selector.scores_,'pvalues':selector.pvalues_})

In [13]:
print(GeneRanks.head())

    Gene   pvalues    scores
0   A1BG  0.421646  0.646983
1   A1CF  0.487033  0.483925
2  A2BP1  0.041865  4.165979
3  A2LD1  0.010812  6.554052
4    A2M  0.793784  0.068416


In [14]:
GeneRanks['pvalue_rank'] = GeneRanks['pvalues'].rank(ascending=True)
#GeneRanks['Fscore_rank'] = GeneRanks['scores'].rank(ascending=False) 
print(GeneRanks.head())

    Gene   pvalues    scores  pvalue_rank
0   A1BG  0.421646  0.646983      13237.0
1   A1CF  0.487033  0.483925      14155.0
2  A2BP1  0.041865  4.165979       5542.0
3  A2LD1  0.010812  6.554052       3503.0
4    A2M  0.793784  0.068416      17903.0


In [19]:
GeneRanks_trim = GeneRanks[GeneRanks['pvalue_rank']<424]
print(GeneRanks_trim[GeneRanks_trim['pvalue_rank'] < 11])

           Gene       pvalues     scores  pvalue_rank
4375    CTNNAL1  3.836709e-12  51.130940          1.0
5576       ENSA  2.446520e-10  42.089129          7.0
6746     GABPB2  2.321568e-10  42.201898          6.0
7834   HIST1H3J  4.899030e-11  45.563846          3.0
10559    MAP6D1  4.745030e-10  40.667103          8.0
12235      OAS1  6.367761e-11  44.995162          4.0
12237      OAS3  8.342875e-12  49.424822          2.0
12709    ORAOV1  2.265594e-10  42.254408          5.0
17993     TIGD7  4.807679e-10  40.639000          9.0
19012    UBE2D2  4.869178e-10  40.611769         10.0


In [17]:
GeneRanks_trim['Gene']

61          ABCC5
292          ADAR
498         AKTIP
529         ALG11
616       ANAPC10
661      ANKRD13D
728        ANP32B
800         APH1A
982       ARL6IP1
986         ARL8A
1015       ARPC5L
1068         ASB6
1258       ATXN2L
1285        AXIN1
1361        BARX1
1456        BEND7
1539         BOD1
1563         BRD2
1638         BUB1
1641        BUD13
1747     C11orf82
1951     C16orf72
1965     C16orf91
1980     C17orf42
1988     C17orf53
2081     C19orf52
2093     C19orf66
2133     C1orf111
2134     C1orf112
2169     C1orf174
           ...   
19709       YWHAZ
19711      YY1AP1
19742      ZBTB34
19750      ZBTB43
19890        ZHX3
19930      ZNF117
19942       ZNF14
19958      ZNF174
19976      ZNF200
19982      ZNF211
20015      ZNF256
20020      ZNF263
20082     ZNF354A
20083     ZNF354B
20089      ZNF367
20115      ZNF418
20116      ZNF419
20127      ZNF433
20128      ZNF434
20147      ZNF467
20194      ZNF530
20313      ZNF681
20314      ZNF682
20323      ZNF695
20354     